In [1]:
%%capture
# !conda remove --all -n gtech-training -y
# !conda create -n gtech-training python==3.11 -y
# !conda activate gtech-training
!pip install -r ../requirements.txt

In [1]:
import torch
torch.cuda.is_available()

True

In [19]:
# import cell
import json
import pandas as pd
from tqdm import tqdm
import sys, os
from torch.utils.data import DataLoader
from transformers import BartTokenizer, BartForConditionalGeneration

In [3]:
my_dir = sys.path[0]
sys.path.append(os.path.join(my_dir, '..'))
from eli5 import *

In [4]:
train_path = '../docs/ELI5.jsonl'
valid_path = '../docs/ELI5_val.jsonl'

In [5]:
with open('../docs/ELI5.jsonl', 'r') as json_file:
    json_list_train = list(json_file)
with open('../docs/ELI5_val.jsonl', 'r') as json_file:
    json_list_valid = list(json_file)

In [6]:
train_dataset = ELI5(json_list_train)
valid_dataset = ELI5(json_list_valid)

In [7]:
bs = 1
train_data_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True)
valid_data_loader = DataLoader(valid_dataset, batch_size=bs, shuffle=False)

In [13]:
# valid_dataset[0]

In [27]:
model_name = "facebook/bart-large"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForQuestionAnswering.from_pretrained(model_name)

Some weights of BartForQuestionAnswering were not initialized from the model checkpoint at facebook/bart-large and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
question_id, question, answers, ctxs = valid_dataset[0]

In [26]:
inputs = tokenizer(question, ctxs[0], return_tensors="pt")

In [28]:
with torch.no_grad():
    outputs = model(**inputs)

In [31]:
answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]

In [32]:
tokenizer.decode(predict_answer_tokens, skip_special_tokens=True)

'J'

In [15]:
valid_dataset[0]

('1kiwfx',
 'In Trading Places (1983, Akroyd/Murphy) how does the scheme at the end of the movie work? Why would buying a lot of OJ at a high price ruin the Duke Brothers?',
 ['The final scene involves future contracts. This simply means entering into a contract to buy something (oil, wheat, even frozen concentrated orange juice(FCOJ)) at a specified time for the current price. The person selling the future does not have to own the FCOJ at the time of sale he simply has to provide them at the agreed upon date. Futures help companies mitigate risk against the unpredictable price of FCOJ. If the price of FCOJ goes up the buyer wins the seller loses and visa versa. This price is often affected by fresh oranges. If there is a good harvest FCOJ price goes down and so on. The Dukes believed there was going to be a bad harvest. Their plan was to buy as much FCOJ as they could and basically corner the market then sell it at a much higher price due to a lack of oranges. So here is what happened